# NLP analysis objectives
- switch to outscraper to extract full information of reviews ✓
- check if the place might have water army?
- extract the most related
- use sentiment analysis to analysis the score rather than using google score
- NLTK

# To Do:
- download a set of data with only reviews and rating or Aurther ✓
- Word2vec to show its true reviews
- sentiment analysis to measure its true value
## data preprocessing:
- stopword ✓
- lemmetize ✓
- porterstemmer (clean liked like liking into like) ✓
- embedding ✓
- train test split ✓
- word2vec after cleaning ✓
- onehotencoding y as well

# Tech issue
- maybe code a function to download necessary nlp trained model

In [1]:
import pandas as pd
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
import numpy as np
import re,string
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /home/aipen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/aipen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/aipen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/aipen/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
df=pd.read_csv('data_references/yelp.csv')
df=df[['text','stars']]

In [3]:
def clean_data(word):
    model= stopwords.words('english')
    trim_s=WordNetLemmatizer()
    stem=PorterStemmer()
    word = ''.join([i for i in str(word) if not i.isdigit()])
    word = word_tokenize(word.lower())
    word = [i for i in set(word) if i.isalpha() and i not in string.punctuation and i not in model]
    word = [trim_s.lemmatize(i) for i in word]
    word = [stem.stem(i) for i in word]
    
    return " ".join(word)

In [4]:
df['true_review']=np.vectorize(clean_data)(df.text)

In [5]:
X=df['true_review']
y=df[['stars']]

 # Hyper parameters for grid search:
## word2vec
- vector_size
- min_count
- window
## padding
- maxlen

In [7]:
vector_size=[50,60,70,80]
window=[5,6,7,8,9,10]
maxlen=[70]

col_vec=[]
col_win=[]
col_max=[]

for a in vector_size:
    for b in window:
        for c in maxlen:
            col_vec.append(a)
            col_win.append(b)
            col_max.append(c)
        

df_test=pd.DataFrame({'vector':col_vec,'window':col_win,'maxlen':col_max})
hist_list=[]
for z in range(0,len(df_test)):
    new_y=[]
    for i in df.stars:
        if i ==5:
            new_y.append([0,0,0,0,1])
        elif i ==4:
            new_y.append([0,0,0,1,0])
        elif i ==3:
            new_y.append([0,0,1,0,0])
        elif i ==2:
            new_y.append([0,1,0,0,0])
        else:
            new_y.append([1,0,0,0,0])
    new_y=np.array(new_y)

    X_train, X_test, y_train, y_test = train_test_split(X, new_y, test_size=0.33, random_state=42)

    word2vec = Word2Vec(sentences=X_train, size=df_test['vector'][z], min_count=1, window=df_test['window'][z])

    from tensorflow.keras.preprocessing.sequence import pad_sequences
    import numpy as np

    # Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
    def embed_sentence(word2vec, sentence):
        embedded_sentence = []
        for word in sentence:
            if word in word2vec.wv:
                embedded_sentence.append(word2vec.wv[word])

        return np.array(embedded_sentence)

    # Function that converts a list of sentences into a list of matrices
    def embedding(word2vec, sentences):
        embed = []

        for sentence in sentences:
            embedded_sentence = embed_sentence(word2vec, sentence)
            embed.append(embedded_sentence)

        return embed

    # Embed the training and test sentences
    X_train_embed = embedding(word2vec, X_train)
    X_test_embed = embedding(word2vec, X_test)

    X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=df_test['maxlen'][z])
    X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=df_test['maxlen'][z])

    from tensorflow.keras import Sequential
    from tensorflow.keras import layers

    def init_model():
        model = Sequential()
        model.add(layers.Masking())
        model.add(layers.LSTM(20, activation='tanh'))
        model.add(layers.Dense(15, activation='relu'))
        model.add(layers.Dense(10, activation='relu'))
        model.add(layers.Dense(5, activation='relu'))
        model.add(layers.Dense(5, activation='softmax'))

        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

        return model
    model=init_model()

    from tensorflow.keras.callbacks import EarlyStopping

    es = EarlyStopping(patience=10, restore_best_weights=True)

    history=model.fit(X_train_pad, y_train, 
              batch_size = 64,
              epochs=500,
              callbacks=[es],
              validation_data=(X_test_pad,y_test),
              verbose=0
             )  
    hist_list.append(history.history['val_accuracy'][-1])
    print(f'finished {z+1} out of {len(df_test)} with {int(((z+1)/len(df_test))*100)}% progress')

2022-07-04 13:32:18.469164: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-04 13:32:18.469198: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-07-04 13:32:24.615125: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-07-04 13:32:24.618928: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-04 13:32:24.619008: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcub

finished 1 out of 24 with 4% progress
finished 2 out of 24 with 8% progress
finished 3 out of 24 with 12% progress
finished 4 out of 24 with 16% progress
finished 5 out of 24 with 20% progress
finished 6 out of 24 with 25% progress
finished 7 out of 24 with 29% progress
finished 8 out of 24 with 33% progress
finished 9 out of 24 with 37% progress
finished 10 out of 24 with 41% progress
finished 11 out of 24 with 45% progress
finished 12 out of 24 with 50% progress
finished 13 out of 24 with 54% progress
finished 14 out of 24 with 58% progress
finished 15 out of 24 with 62% progress
finished 16 out of 24 with 66% progress
finished 17 out of 24 with 70% progress
finished 18 out of 24 with 75% progress
finished 19 out of 24 with 79% progress
finished 20 out of 24 with 83% progress
finished 21 out of 24 with 87% progress
finished 22 out of 24 with 91% progress
finished 23 out of 24 with 95% progress
finished 24 out of 24 with 100% progress


In [8]:
df_test['val_acc']=hist_list

In [9]:
df_test

,vector,window,maxlen,val_acc
0,50,5,70,0.363636
1,50,6,70,0.353636
2,50,7,70,0.363636
3,50,8,70,0.362727
4,50,9,70,0.348788
5,50,10,70,0.357576
6,60,5,70,0.377576
7,60,6,70,0.368485
8,60,7,70,0.356667
9,60,8,70,0.372727


now need to convert 

In [ ]:

!git add NLP.ipynb
!git commit -m 'created function for grid search'
!git push origin master


### 